<a href="https://colab.research.google.com/github/AyishaR/Spokendigit/blob/master/Spokendigit_MFCCs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
from torch.utils.data import Dataset, random_split, DataLoader, TensorDataset
import torchvision
from torchvision.datasets.utils import download_url
import torch.nn as nn
import torch.nn.functional as F
import tarfile
import os
import librosa
import pandas as pd
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import librosa.display
import sklearn
import matplotlib
import csv
from PIL import Image
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score

#Data

In [4]:
digit = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']

# MFCCs

Spoken_digit_finalfts.csv generated in 'Spokendigit_feature_extraction.ipynb'.

In [7]:
finalfts = pd.read_csv('Spoken_digit_finalfts.csv')
spokendset = TensorDataset(torch.tensor(np.array(finalfts[finalfts.columns[:20]].astype('float32'))),torch.tensor(finalfts['Label'])) 
size = len(spokendset)
val_size = int(0.1 * size)
train_size = size - val_size 
train_dset, val_dset = random_split(spokendset, [train_size, val_size])

print(train_size, val_size)

train_dl = DataLoader(train_dset, 64, True, num_workers=6, pin_memory=True)
val_dl = DataLoader(val_dset, 64, num_workers=6, pin_memory=True)

21298 2366


#device


In [8]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

In [9]:
device = get_default_device()
device

device(type='cpu')

In [10]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)

#Train

In [11]:
class SpokenDigitModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(20, 1024)
        self.l2 = nn.Linear(1024, 512)
        self.l3 = nn.Linear(512, 64)
        self.l4 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = self.l4(x)
        return x

    def training_step(self, batch):
        inputs, labels = batch
        outputs = self(inputs)
        loss = F.cross_entropy(outputs, labels)
        return loss

    def validation_step(self, batch):
        inputs, labels = batch
        outputs = self(inputs)
        loss = F.cross_entropy(outputs, labels)
        _, pred = torch.max(outputs, 1)
        accuracy = torch.tensor(torch.sum(pred==labels).item()/len(pred))
        return [loss.detach(), accuracy.detach()] 

In [20]:
def evaluate(model, loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in loader]
    outputs = torch.tensor(outputs).T
    loss, accuracy = torch.mean(outputs, dim=1)
    return {"loss" : loss.item(), "accuracy" : accuracy.item()}

In [13]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [14]:
def fit(model, train_loader, val_loader, epochs, lr, optimizer_function = torch.optim.Adam):
    history = []
    optimizer = optimizer_function(model.parameters(), lr)
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, lr, epochs=epochs, steps_per_epoch=len(train_loader))
    for epoch in range(epochs):
        print("Epoch ", epoch)
        #Train
        model.train()
        lrs = []
        tr_loss = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            tr_loss.append(loss)
            loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()
            lrs.append(get_lr(optimizer))
            sched.step()
        #Validate
        result = evaluate(model, val_loader)
        result["lrs"] = lrs
        result["train loss"] = torch.stack(tr_loss).mean().item()
        
        print("Last lr: ", lrs[-1]," Train_loss: ", result["train loss"], " Val_loss: ", result['loss'], " Accuracy: ", result['accuracy'])
        history.append(result)         
    return history

In [21]:
model = to_device(SpokenDigitModel(), device)
history = []
evaluate(model, val_dl)

{'accuracy': 0.08834186941385269, 'loss': 8.290132522583008}

In [22]:
history.append(fit(model, train_dl, val_dl, 128, 0.001))

Epoch  0



Last lr:  4.159611188916403e-05  Train_loss:  2.0483174324035645  Val_loss:  1.7843549251556396  Accuracy:  0.4117670953273773
Epoch  1



Last lr:  4.6393002654010734e-05  Train_loss:  1.6750222444534302  Val_loss:  1.6015233993530273  Accuracy:  0.4433985650539398
Epoch  2



Last lr:  5.435859286315734e-05  Train_loss:  1.495118260383606  Val_loss:  1.3963572978973389  Accuracy:  0.5309094786643982
Epoch  3



Last lr:  6.54395881999855e-05  Train_loss:  1.3480528593063354  Val_loss:  1.2373576164245605  Accuracy:  0.5710412859916687
Epoch  4


...


Last lr:  4.913417694027851e-06  Train_loss:  0.00025864565395750105  Val_loss:  0.29073935747146606  Accuracy:  0.9556179642677307
Epoch  124



Last lr:  2.7675290714544136e-06  Train_loss:  0.0002540568239055574  Val_loss:  0.2924289405345917  Accuracy:  0.9568848609924316
Epoch  125



Last lr:  1.2328645375991738e-06  Train_loss:  0.0002496350498404354  Val_loss:  0.291685551404953  Accuracy:  0.956462562084198
Epoch  126



Last lr:  3.1131057456677687e-07  Train_loss:  0.00024712487356737256  Val_loss:  0.2913511097431183  Accuracy:  0.9560402631759644
Epoch  127



Last lr:  4e-09  Train_loss:  0.00024529328220523894  Val_loss:  0.29120033979415894  Accuracy:  0.9556179642677307


In [23]:
@torch.no_grad()
def predict_dl(model, dl):
    torch.cuda.empty_cache()
    batch_probs = []
    batch_targ = []
    for xb, yb in dl:
        probs = model(xb)
        batch_probs.append(probs.cpu().detach())
        batch_targ.append(yb.cpu().detach())
    batch_probs = torch.cat(batch_probs)
    batch_targ = torch.cat(batch_targ)
    return [list(values).index(max(values)) for values in batch_probs], batch_targ

In [24]:
r = evaluate(model, val_dl)
yp, yt = predict_dl(model, val_dl)
print("Loss: ", r['loss'], "\nAccuracy: ", r['accuracy'], "\nF-score: ", f1_score(yt, yp, average='micro'))

Loss:  0.29120033979415894 
Accuracy:  0.9556179642677307 
F-score:  0.9556213017751479


In [25]:
torch.save(model, '/content/spokendigit_lr_mfccs.pth')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type SpokenDigitModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
